In [1]:
import pandas as pd

data = {
    'CSV': ['', '', '', ''],
    'country': ['Australia', 'Australia', 'Australia', 'Australia'],
    'tab': ['Freight transport', 'Freight transport', 'Freight transport', 'Freight transport'],
    'flow': ['SHARE_COUNTRY_EM', 'SHARE_COUNTRY_EM', 'SHARE_FOSSIL', 'SHARE_FOSSIL'],
    'enduse': ['Freight trains', 'Trucks', 'Domestic freight ships', 'Freight trains'],
    'activity': ['', '', '', ''],
    'unit': ['%', '%', '%', '%'],
    'year': [2013.0, 2013.0, 2013.0, 2013.0],
    'value': [0.9, 8.65, 88.52, 94.43]
}

df = pd.DataFrame(data)

pivoted_df = df.pivot_table(index=['flow', 'country',  'tab', 'enduse'], columns='year', values='value', aggfunc='first').reset_index().rename_axis(None, axis=1)

pivoted_df

,flow,country,tab,enduse,2013.0
0,SHARE_COUNTRY_EM,Australia,Freight transport,Freight trains,0.90
1,SHARE_COUNTRY_EM,Australia,Freight transport,Trucks,8.65
2,SHARE_FOSSIL,Australia,Freight transport,Domestic freight ships,88.52
3,SHARE_FOSSIL,Australia,Freight transport,Freight trains,94.43


In [2]:
import pandas as pd
import numpy as np

# Example DataFrame with 10 columns
df = pd.DataFrame({
    'A': [np.nan, 2, 3],
    'B': [4, np.nan, np.nan],
    'C': [np.nan, np.nan, 7],
    'D': [8, 9, np.nan],
    'E': [np.nan, 11, 12],
    'F': [13, np.nan, np.nan],
    'G': [np.nan, np.nan, 16],
    'H': [17, np.nan, np.nan],
    'I': [np.nan, 19, 20],
    'J': [21, np.nan, np.nan]
})

# Function to fill each column with the first non-null value
def fill_first_non_null(column):
    first_non_null = column.first_valid_index()
    return column.fillna(column[first_non_null])

# Create the resulting DataFrame
result = df.apply(fill_first_non_null).head(1)



result

,A,B,C,D,E,F,G,H,I,J
0,2.0,4.0,7.0,8.0,11.0,13.0,16.0,17.0,19.0,21.0


In [3]:
import pandas as pd

languages = ["en", "fr", "es"] #, 'zh']  # List of languages

# Initialize an empty dataframe
df_merged = pd.DataFrame()
info_dir = '../../data/_info'  
info_file = '__INFO_UN_M49'

# Loop through the languages
for lang in languages:
    # Read the CSV file for the current language
    filename = f"{info_dir}/{info_file}_{lang}.csv"
    df_language = pd.read_csv(filename)
    
    # Merge the current language dataframe with the merged dataframe
    if df_merged.empty:
        df_merged = df_language
    else:
        df_merged = pd.merge(df_merged, df_language, on="Code", how="outer")

# Set the language code column as the index
df_merged.set_index("Code", inplace=True)

# Rename the columns to the respective language names
column_names = {"column_name_en": "English",
                "column_name_fr": "French",
                "column_name_es": "Spanish",
                "column_name_zh": "Chinese"}
df_merged.rename(columns=column_names, inplace=True)

# Save the resulting dataframe as a CSV file
df_merged.to_csv(f"{info_dir}/{info_file}.csv")

# Display a message upon successful completion
print("Merged dataframe saved as 'merged_languages.csv'")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/_info/__INFO_UN_M49_en.csv'

In [ ]:

import os
import pandas as pd
import io
import re
import numpy as np

exclude_columns = [
    'iso3166_numeric'
    
]

# Function to fill each column with the first non-null value
def first_non_null(column):
    first_non_null = column.first_valid_index()
    return column.fillna(column[first_non_null])

# Function to replace a substring in a string column
def process_values(value):   
    if isinstance(value, str):
        value = re.sub(r'[\r\n]+', ' - ', value).strip('"\' \)\()')
    
    return np.nan if not value else value


def exclude(column_name):
    if column_name in exclude_columns:
        return True
    if str(column_name).startswith('dataflag'):
        return True

def process_columns(column_name):
    #return column_name
    
    # BASIC 
    column_name = re.sub(r'([A-Z]+)', r' \1', column_name.lower())
    column_name = re.sub(r'(\s+|\.)', r' ', column_name).strip()
    column_name = re.sub(r'(\s+)', r'_', column_name)   
    
    # COUNTRY AND REGION 
    column_name = re.sub(r'(iso[2-3])code', r'\1', column_name)
    column_name = re.sub(r'(country)(iso[2-3])', r'\1_\2', column_name)
    column_name = re.sub(r'(country)(region)', r'\2', column_name)
    column_name = re.sub(r'(?<=region)(?=type|name)', '_', column_name)
    column_name = re.sub(r'(?<=region)(?=id)', '_', column_name)
    column_name = re.sub(r'(?<=type)(?=id|name)', '_', column_name)

    
    # WB specific
    column_name = re.sub(r'(?<=source)(?=organization|note)', '_', column_name)
    column_name = re.sub(r'(?<=data)(?=availability)', '_', column_name)
    
    # EIA specific
    if (column_name=='period'):
        column_name = column_name.replace('period','year')   
            
    # UNIT
    column_name = re.sub(r'(?<=unit)(?=name)', '_', column_name)
    
    # MISC
    if (column_name=='var'):
        column_name = column_name.replace('var','variable')
            
    # UID
    if (column_name=='code'):
        column_name = re.sub(r'code', 'uid', column_name) 
    #column_name = column_name.replace('country_id','country_iso2')  
    column_name = column_name.replace('country_code','country_iso3')  
 
    # DATE RELATED    
    column_name = re.sub(r'(?<=last)(?=updated)', '_', column_name) 
    
    return column_name

# FORMAT DF: 
def process_df(df):
    # PROCESSING 
    # STRING VALUES (might set to NaN)
    df = df.applymap(process_values) 
    # Uniformize NaN values
    df = df.replace(['', 'nan'], np.nan)  
                
    # Drop all columns having only NaN
    df = df.dropna(axis=1, how='all')  

    # Replace column names using the format function
    df = df.rename(columns=process_columns)

    # Filter value if exists
    if ('value' in df.columns):
        df = df.dropna(subset=['value'])

    # DATE: Split the 'date' using regex if it matches the pattern
    # Date column format: YYYY-MM-DD OR YYYY
    if ('date' in df.columns):
        df[['year', 'month']] = df['date'].astype(str).str.extract(r'^(\d{4})-(\d{2})-\d{2}$').fillna(np.nan)
        # Drop all columns having only NaN AGAIN due to previously added columns
        df = df.dropna(axis=1, how='all') 
        
        if ('year' in df.columns):
            df = df.drop(columns=['date'])
        else:
            # Date is in YYYY format
            df = df.rename(columns={'date': 'year'})   
    
    if ('year' in df.columns):
            df['year']=df['year'].astype(int)
            if ('month' in df.columns):
                df['month']=df['month'].astype(int)    
    
    # Unit
    if ('unit' in df.columns and not 'unit_name' in df.columns ):
        df['unit_name']=df['unit']  
    
    # WB SPECIFIC    
    # Manage inconsistent country/region iso2 or iso3 in indicators
    if ('country_id' in df.columns and 'country_value' in df.columns ):
        if ('country_iso3' in df.columns):
            df['country_iso3'] = df['country_iso3'].fillna(df['country_id'])
        else:
            df['country_iso3']=df['country_id']
        df = df.drop(columns=['country_id','country_value'])
            
    return df


# Create an empty set to store unique column names
data_columns = set()
info_columns = set()
all_df = pd.DataFrame()
all_rows=0

# 
info_dir = '../../data/_raw'  
for root, dirs, _ in os.walk(info_dir):
    for dir in dirs:
        # Iterate over files within each subdirectory
        source_dir = os.path.join(root, dir)
        
        print(f'\nRAW SOURCE: {dir.upper()}') 
                       
        info_df = pd.DataFrame()
        data_df = pd.DataFrame()
        # to check
        nb_rows=0

        has_info = False
        for file in os.listdir(source_dir):

            info  =False
            # Perform operations on each file          
            if file.endswith(".csv"):

                # Read only the first line of the CSV file
                csv_path =  os.path.join(source_dir, file)
                csv_name = os.path.splitext(os.path.basename(csv_path))[0]
               
                print(f'CSV NAME: {csv_path}')
                info = file.startswith("__info")
                print(f'INFO : {info}')
                                
                # Build df
                df = pd.read_csv(csv_path, low_memory=False) 
                
                ## PROCESSING
                df = process_df(df)
                
                nb_rows += 0 if info else len(df)
                all_rows += nb_rows

                print(sorted(list(df.columns)))                                

                #output_df = info_df if info else data_df
                # Append the current iteration DataFrame to the output DataFrame
                if (info) :
                    info_columns.update(df.columns)
                    info_df = pd.concat([info_df, df], ignore_index=True)
                else:
                    data_columns.update(df.columns)
                    data_df = pd.concat([data_df, df], ignore_index=True)
                        
        print(f'\nExpected nb rows for {dir.upper()}: {nb_rows}')
        print(f'ACTUAL nb rows for {dir.upper()}: {len(data_df)}')
        
        if (not info_df.empty):
            info_df = info_df.drop_duplicates()
            info_df['RAW']=dir.upper()
            info_df.to_csv(f'_INFO_{dir.upper()}.csv', index=False)
                        
        if (not data_df.empty):
            data_df = data_df.drop_duplicates()
            data_df['RAW']=dir.upper()
            data_df.to_csv(f'PROCESSED_{dir.upper()}.csv', index=False)

            
            # ONLY FOR TEST
            # Create the resulting DataFrame
            result = data_df.apply(fill_first_non_null).head(1)
            #selected_columns = list(set(output_df.columns).intersection(column_names))
            # # Step 3: Create the output DataFrame with all columns from the exhaustive list
            #all_pdf = pd.DataFrame(columns=["CSV"] + column_names)
            #all_df[selected_columns] = output_df[selected_columns].head(100)
            
            all_df = pd.concat([all_df,  result], ignore_index=True)

print(f'\nExpected nb rows for ALL: {all_rows}')
print(f'ACTUAL ALL DF nb rows: {len(all_df)}')
all_df.to_csv(f'PROCESSED__ALL.csv', index=False)        
# Convert the set to a list and sort it
info_columns = sorted(list(info_columns))
print(f'INFO NB COLS {len(info_columns)}')
print(f'INFO COLS {info_columns}')

data_columns = sorted(list(data_columns))
print(f'DATA NB COLS {len(data_columns)}')
print(f'DATA COLS {data_columns}')





In [ ]:
all_df = all_df.sort_index(axis=1)
# Select the first non-null value from each column
sample_row = all_df.apply(fill_first_non_null).head(1)


sample_row

In [ ]:
import pandas as pd
import os
import numpy as np

csv_file_path = "../../data/_raw/ember/ember_file_elec_review_global_month.csv"
csv_file_name = os.path.splitext(os.path.basename(csv_file_path))[0]  # Extract CSV file name
df = pd.read_csv(csv_file_path)

# Uniformize NaN
df = df.replace(['', 'nan'], np.nan)
# Drop all columns having only NaN
df = df.dropna(axis=1, how='all')  

# Replace column names using the format function
df = df.rename(columns=process_columns)

# Filter value if exists
if ('value' in df.columns):
    df = df.dropna(subset=['value'])




# DATE: Split the 'date' using regex if it matches the pattern
# Date column format: YYYY-MM-DD OR YYYY
if ('date' in df.columns):
    df[['year', 'month']] = df['date'].astype(str).str.extract(r'^(\d{4})-(\d{2})-\d{2}$').fillna(np.nan)    

# Drop all columns having only NaN AGAIN due to previously added columns
df = df.dropna(axis=1, how='all') 
if ('year' in df.columns):
    df = df.drop('date', axis=1)
else:
    # Date is in YYYY format
    df = df.rename(columns={'date': 'year'})
 

print(list(df.columns))
df
# selected_columns = list(set(df_csv.columns).intersection(column_names))

# # Step 3: Create the output DataFrame with all columns from the exhaustive list
# output_df = pd.DataFrame(columns=["CSV"] + column_names)

# # Step 4: Fill the matching columns from the CSV file in the output DataFrame
# output_df[selected_columns] = df_csv[selected_columns]


# output_df["CSV"] = csv_file_name
# output_df = output_df.dropna(axis=1, how='all')
# print(len(list(output_df.columns)))

# output_df

In [5]:
# %pip install rapidfuzz
# %pip install fuzzywuzzy

# import pandas as pd
# from rapidfuzz import fuzz as rfuzz
# from rapidfuzz import process as rprocess
# from fuzzywuzzy import fuzz as ffuzz
# from fuzzywuzzy import process as fprocess
# import re

# df=output_df
# # Perform fuzzy matching on the column names list using rapidfuzz
# target_column = "region"
# threshold = 0  # Adjust the threshold as per your requirements

# matches_rfuzz = rprocess.extract(target_column, df['fuzzy'])
# matched_columns_rfuzz = [match[0] for match in matches_rfuzz if match[1] >= threshold]
# matched_columns_rfuzz = sorted(list(set(matched_columns_rfuzz)))

# filtered_values_rfuzz = output_df.loc[df["fuzzy"].isin(matched_columns_rfuzz), "column"].tolist()

# #print("RapidFuzz Matches:")
# #print(matched_columns_rfuzz)

# # Perform fuzzy matching on the column names list using fuzzywuzzy
# matches_ffuzz = fprocess.extract(target_column, df['fuzzy'])
# matched_columns_ffuzz = [match[0] for match in matches_ffuzz if match[1] >= threshold]
# matched_columns_ffuzz = sorted(list(set(matched_columns_ffuzz)))

# filtered_values_ffuzz = df.loc[df["fuzzy"].isin(matched_columns_ffuzz), "column"].tolist()

# #print("\nFuzzyWuzzy Matches:")
# #print(matched_columns_ffuzz)

# # Combine the matches from both libraries and remove duplicates
# all_matches = sorted(list(set(filtered_values_rfuzz + filtered_values_ffuzz)))

# print("\nMatches")
# print(all_matches)
